In [1]:
! pip install category_encoders

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from category_encoders import *
from sklearn.preprocessing import *
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import *
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import *

import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Table of Contents

* [Background](#Background)
    * [Load in data](#Load_in_data)
        * [Encode target](#Encode_target)
        * [Missing values](#Missing_values)   
        * [Imbalanced data](#Imbalanced_data)
    * [Split data](#Split_data)
    * [Metrics](#Metrics)
    * [Choose algorithms](#Choose_algorithms)
        
* [Logistic Regression](#LogisticRegression)
    * [SMOTE](#lr_smote)
    * [Column transformations](#lr_encoding)
        * [Categorical](#lr_categorical)
        * [Numeric](#lr_continuous) 
        * [Combined](#lr_combined_encoders) 
    * [Tune hyperparameters](#lr_hyperparameters)

* [Random Forest](#RandomForest)
    * [SMOTE](#rf_smote)
    * [Column transformations](#rf_encoding)
        * [Categorical](#rf_categorical)
        * [Numeric](#rf_continuous) 
        * [Combined](#rf_combined_encoders) 
    * [Tune hyperparameters](#rf_hyperparameters)

* [Final Model](#Final_model)
    * [Conclusion](#Conclusion)
    * [Future directions](#Future_directions)

### Background

The data used here comes from the UCI Machine Learning Repository; it is the "Drug consumption (quantified)" dataset. The dataset can be found at [this url.](https://archive.ics.uci.edu/ml/datasets/Drug+consumption+%28quantified%29#)

The data itself was collected via online survery from 1885 participants. The data collected includes demographic information, such as age, gender, education, country of residence, and ethnicity, personality inventory scores on traits such as neuroticism, extraversion, openness to experience, agreeableness, conscientiousness, impulsivity, and sensation seeking. In addition to these attributes, level of consumption of 18 drugs such as alcohol, amphetamines, amyl nitrite, benzodiazepine, cannabis, chocolate, cocaine, caffeine, crack, ecstasy, heroin, ketamine, legal highs, LSD, methadone, mushrooms, nicotine and volatile substance abuse were collected. To disqualify any potential participants who wanted to over-report use of drugs, participants also reported their consumption of a fake drug called "Semeron". 

Participants were given 7 levels of consumption to choose from for each drug, "Never Used", "Used over a Decade Ago", "Used in Last Decade", "Used in Last Year", "Used in Last Month", "Used in Last Week", and "Used in Last Day".

In this notebook, I've decided to try to predict the level of participants' use of cannabis based on the collected demographic and personality inventory scores using machine learning. Here, I try two different machine learning algorithms to predict level of cannabis use: a logistic regression classifier model and a random forest classifier model.

** Note that the original data on the UCI website has been transformed from categorical values to a scaled numeric value, I simply converted the scaled numeric values back to their original categorical values and use the converted data in this notebook.

### Load in data <a class="anchor" id="Load_in_data"></a>

In [3]:
url = 'https://raw.githubusercontent.com/audreybarszcz/drug_consumption_machine_learning/main/drug_consumption.csv'
data = pd.read_csv(url)
data.head()

,ID,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SS,Alcohol,Amphet,Amyl,Benzos,Caff,Cannabis,Choc,Coke,Crack,Ecstasy,Heroin,Ketamine,Legalh,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,1,35-44,Female,Professional certificate/ diploma,UK,Mixed-White/Asian,39.0,36.0,42.0,37.0,42.0,-0.21712,-1.18084,CL5,CL2,CL0,CL2,CL6,CL0,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,2,25-34,Male,Doctorate degree,UK,White,29.0,52.0,55.0,48.0,41.0,-0.71126,-0.21575,CL5,CL2,CL2,CL0,CL6,CL4,CL6,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,3,35-44,Male,Professional certificate/ diploma,UK,White,31.0,45.0,40.0,32.0,34.0,-1.37983,0.40148,CL6,CL0,CL0,CL0,CL6,CL3,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,4,18-24,Female,Masters degree,UK,White,34.0,34.0,46.0,47.0,46.0,-1.37983,-1.18084,CL4,CL0,CL0,CL3,CL5,CL2,CL4,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,5,35-44,Female,Doctorate degree,UK,White,43.0,28.0,43.0,41.0,50.0,-0.21712,-0.21575,CL4,CL1,CL1,CL0,CL6,CL3,CL6,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0


In [4]:
X = data.iloc[:, 1:13] # feature columns, ignoring ID column
y = data.iloc[:, 18]   # cannabis use column

#### Encode target <a class="anchor" id="Encode_target"></a>

In [5]:
# transform y from string categories to numeric values
le = LabelEncoder().fit(y)
y = le.transform(y)

#### Missing values <a class="anchor" id="Missing_values"></a>

In [6]:
# luckily we have no missing values to impute!
data.isna().sum().sum()

0

#### Imbalanced data <a class="anchor" id="Imbalanced_data"></a>
Looking at the distribution of class labels y, there may be some class imbalance. For example, the class 0 (Never Used) and class 6 (Used in the last day) labels occur 2-3 times more often than classes 4 (Used in last month) or 5 (Used in last week). In an effort to make our predictions better, try adding SMOTE to our pipeline.

In [7]:
# relative proportion of each label
data.groupby('Cannabis').count()['Age']/data.groupby('Cannabis').count()['Age'].sum()

Cannabis
CL0    0.219098
CL1    0.109814
CL2    0.141114
CL3    0.111936
CL4    0.074271
CL5    0.098143
CL6    0.245623
Name: Age, dtype: float64

### Split data <a class="anchor" id="Split_data"></a>

I chose 0.15 to use as the test data, leaving 0.85 remaining to split into training and validation sets. I again used a split of 0.15 for validation, 12.75% of the original data, and 0.85 for training, 72.25% of the original data. I wanted to split the data such that I would have about 75% of the data to train on.

In [8]:
# split off test data and never use until the end!!!
X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), test_size=0.15)

In [9]:
# split training set again into training and validation set
X_train_train, X_val, y_train_train, y_val = train_test_split(X_train, y_train, test_size=0.15)

### Metrics <a class="anchor" id="Metrics"></a>

For evaluating performance of each model, I am looking at both balanced accuracy score and the F1-weighted score. I feel that the F1-weighted score is a more accurate measure of performance of my model since it takes into account the class weights in addition to considering both precision and recall. Whereas, the balanced accuracy score simply averages the recall scores of each class, weighting each class equally. Therefore, I use F1-weighted as my scoring metric in the RandomizedSearchCV. 

In [10]:
# helpful for trying different algorithms
class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass
    

# helpful for trying different encoders/scalers
class DummyTransformer(TransformerMixin):
    def fit(self): pass
    def transform(self): pass


def fit_and_predict_random(grid, iterations):
    # search for best tranformer/algorithm
    # print out its scores and parameters
    rand = RandomizedSearchCV(estimator=pipe, 
                              param_distributions=grid, 
                              n_iter=iterations,
                              cv=5, 
                              scoring='f1_weighted',
                              n_jobs=-1,
                              verbose=1)

    best_model = rand.fit(X_train_train, y_train_train)
    y_pred = rand.predict(X_val)
    print('Balanced Accuracy:', balanced_accuracy_score(y_val, y_pred))
    print('F1 weighted:', f1_score(y_val, y_pred, average='weighted'))
    print(best_model.best_estimator_.get_params()['random'])
    pass


def fit_predict_pipe(X_train, y_train, X_test, y_test, pipe):
    # fit the pipe, get predictions and print out metrics
    model = pipe.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('Balanced Accuracy:', balanced_accuracy_score(y_test, y_pred))
    print('F1 weighted:', f1_score(y_test, y_pred, average='weighted'))
    pass


def best_random_column(algorithm, grid, iterations, column):
    # try different transformers for different columns, using respective model
    pipe = Pipeline([('random', DummyTransformer()), 
                     ('model', algorithm)])

    rand = RandomizedSearchCV(estimator=pipe, 
                              param_distributions=grid, 
                              n_iter=iterations,
                              cv=5, 
                              scoring='f1_weighted',
                              n_jobs=-1,
                              verbose=1)
    
    if X_train_train.iloc[:, column].dtype == 'float64':
        best_model = rand.fit(X_train_train.iloc[:, column].values \
                              .reshape(-1, 1), y_train_train)
        
    else:
        best_model = rand.fit(X_train_train.iloc[:, column], y_train_train)
    print(best_model.best_estimator_.get_params()['random'])
    pass


def best_random_column_tuning(algorithm, transformer, grid, iterations, column):
    # try different transformer hyperparameters for different columns, using respective model
    pipe = Pipeline([('random', transformer), 
                     ('model', algorithm)])

    rand = RandomizedSearchCV(estimator=pipe, 
                              param_distributions=grid, 
                              n_iter=iterations,
                              cv=5, 
                              scoring='f1_weighted',
                              n_jobs=-1,
                              verbose=1)
    
    if X_train_train.iloc[:, column].dtype == 'float64':
        best_model = rand.fit(X_train_train.iloc[:, column].values \
                              .reshape(-1, 1), y_train_train)
        
    else:
        best_model = rand.fit(X_train_train.iloc[:, column], y_train_train)
    print(best_model.best_estimator_.get_params()['random'])
    pass

### Choose algorithms <a class="anchor" id="Choose_algorithms"></a>

I wanted to try two different models from sklearn, one from outside the ensemble module and one from the ensemble module.

In [11]:
# not including algorithms from sklearn.ensemble module
algorithms = [{'random': [BernoulliNB()]},
              {'random': [DecisionTreeClassifier()]},
              {'random': [ExtraTreeClassifier()]},
              {'random': [GaussianNB()]},
              {'random': [LogisticRegression(multi_class='multinomial', max_iter=1000)]},
              {'random': [RidgeClassifier()]}]

In [12]:
# basic bare minimum scaling/encoding
categorical_cols = (X.dtypes == 'object')

categorical_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore'))])

continuous_pipe = Pipeline([('scaler', StandardScaler())])

preprocessing = ColumnTransformer([('categorical', categorical_pipe,  categorical_cols),
                                   ('continuous',  continuous_pipe, ~categorical_cols)])

pipe = Pipeline([('preprocessing', preprocessing), 
                 ('random', DummyEstimator())])

In [13]:
# search for best basic algorithm from outside ensemble module
fit_and_predict_random(algorithms, 6)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:    2.4s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    2.5s finished


Balanced Accuracy: 0.29748756247203456
F1 weighted: 0.33522357656205837
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [14]:
# best model among ensemble module
algorithms = [{'random': [ExtraTreesClassifier()]},
              {'random': [RandomForestClassifier()]}]

In [15]:
# search for best basic algorithm from outside ensemble module
fit_and_predict_random(algorithms, 2)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.5s finished


Balanced Accuracy: 0.2816305990995432
F1 weighted: 0.3207117088631001
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


### LogisticRegression

In [16]:
# baseline LogisticRegression performance without SMOTE
pipe = Pipeline([('preprocess', preprocessing),
                 ('lr', LogisticRegression(multi_class='multinomial', max_iter=1000))])

fit_predict_pipe(X_train_train, y_train_train, X_val, y_val, pipe)

Balanced Accuracy: 0.29748756247203456
F1 weighted: 0.33522357656205837


#### SMOTE <a class="anchor" id="lr_smote"></a>

In [17]:
# try adding SMOTE to LogisticRegression
pipe = Pipeline([('preprocess', preprocessing),
                 ('random', SMOTE()), 
                 ('lr', LogisticRegression(multi_class='multinomial', max_iter=1000))])

In [18]:
# hyperparameter grid for SMOTE
strategy = ['minority', 'not minority', 'not majority', 'all', 'auto']
neighbors = [1, 3, 5, 7, 9]

hyperparameters = [{'random': [SMOTE()],
                    'random__sampling_strategy': strategy,
                    'random__k_neighbors': neighbors}]

In [19]:
# adding does not seem to help LogisticRegression
fit_and_predict_random(hyperparameters, 25)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:   28.3s finished


Balanced Accuracy: 0.2730686282316717
F1 weighted: 0.3131196417251318
SMOTE(k_neighbors=7, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
      out_step='deprecated', random_state=None, ratio=None,
      sampling_strategy='minority', svm_estimator='deprecated')


#### Column Transformations <a class="anchor" id="lr_encoding"></a>
As a baseline, I am just using one-hot encoding for all categorical columns and StandardScaler for all numeric columns. Maybe different encodings could make the models better.

##### Categorical columns <a class="anchor" id="lr_categorical"></a>

In [20]:
# columns 0-4 are categorical
categorical_cols

Age           True
Gender        True
Education     True
Country       True
Ethnicity     True
Nscore       False
Escore       False
Oscore       False
Ascore       False
Cscore       False
Impulsive    False
SS           False
dtype: bool

In [21]:
# baseline prediction for LogisticRegression only using categorical columns
# categorical variables should only be one hot encoded for linear models
pipe = Pipeline([('ohe', OneHotEncoder()), 
                 ('lr', LogisticRegression(max_iter=1000, multi_class='multinomial'))])

lr_cat_model = pipe.fit(X_train_train.iloc[:, :4], y_train_train)
y_pred = lr_cat_model.predict(X_val.iloc[:, :4])
print('Balanced Accuracy:', balanced_accuracy_score(y_val, y_pred))
print('F1 weighted:', f1_score(y_val, y_pred, average='weighted'))

Balanced Accuracy: 0.2353828710971568
F1 weighted: 0.25130735345526656


##### Numeric columns <a class="anchor" id="lr_continuous"></a>

In [22]:
# columns 5-12 are numeric
~categorical_cols

Age          False
Gender       False
Education    False
Country      False
Ethnicity    False
Nscore        True
Escore        True
Oscore        True
Ascore        True
Cscore        True
Impulsive     True
SS            True
dtype: bool

In [23]:
# baseline prediction for LogisticRegression only using continuous columns
pipe = Pipeline([('scaler', StandardScaler()), 
                 ('lr', LogisticRegression(max_iter=1000, multi_class='multinomial'))])

lr_cont_model = pipe.fit(X_train_train.iloc[:, 5:], y_train_train)
y_pred = lr_cont_model.predict(X_val.iloc[:, 5:])
print('Balanced Accuracy:', balanced_accuracy_score(y_val, y_pred))
print('F1 weighted:', f1_score(y_val, y_pred, average='weighted'))

Balanced Accuracy: 0.2306791092505378
F1 weighted: 0.23222104614703948


In [24]:
# possible continuous scalers
cont_encoders = [{'random': [StandardScaler()]}, 
                 {'random': [MinMaxScaler()]}, 
                 {'random': [MaxAbsScaler()]},  
                 {'random': [RobustScaler()]}, 
                 {'random': [QuantileTransformer()]}, 
                 {'random': [KBinsDiscretizer()]}]

In [25]:
# find best encoder for Nscore column
best_random_column(LogisticRegression(max_iter=1000, multi_class='multinomial'), cont_encoders, 6, 5)
# find best encoder for Escore column
best_random_column(LogisticRegression(max_iter=1000, multi_class='multinomial'), cont_encoders, 6, 6)
# find best encoder for Oscore column
best_random_column(LogisticRegression(max_iter=1000, multi_class='multinomial'), cont_encoders, 6, 7)
# find best encoder for Ascore column
best_random_column(LogisticRegression(max_iter=1000, multi_class='multinomial'), cont_encoders, 6, 8)
# find best encoder for Cscore column
best_random_column(LogisticRegression(max_iter=1000, multi_class='multinomial'), cont_encoders, 6, 9)
# find best encoder for impulsive column
best_random_column(LogisticRegression(max_iter=1000, multi_class='multinomial'), cont_encoders, 6, 10)
# find best encoder for SS column
best_random_column(LogisticRegression(max_iter=1000, multi_class='multinomial'), cont_encoders, 6, 11)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


StandardScaler(copy=True, with_mean=True, with_std=True)
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KBinsDiscretizer(encode='onehot', n_bins=5, strategy='quantile')
Fitting 5 folds for each of 6 candidates, totalling 30 fits


...

[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


MaxAbsScaler(copy=True)
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


StandardScaler(copy=True, with_mean=True, with_std=True)
Fitting 5 folds for each of 6 candidates, totalling 30 fits
QuantileTransformer(copy=True, ignore_implicit_zeros=False, n_quantiles=1000,
                    output_distribution='uniform', random_state=None,
                    subsample=100000)


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.7s finished


For columns that the best scaler is QuantileTransformer or KBinsDiscretizer, try tuning hyperparameters.

In [26]:
# hyperparameter grid for QuantileTransformer
nquantiles = [50, 100, 150, 200, 250]
dist = ['uniform', 'normal']

quant_hyperparams = [{'random': [QuantileTransformer()]}, 
                     {'random__n_quantiles': nquantiles}, 
                     {'random__output_distribution': dist}]

In [27]:
# hyperparameter grid for KBinsDiscretizer
nbins = [3, 5, 7, 9, 11]
encoding = ['one-hot', 'ordinal']
strategy = ['uniform', 'quantile', 'kmeans']

kbins_hyperparams = [{'random': [KBinsDiscretizer()]}, 
                     {'random__n_bins': nbins}, 
                     {'random__encode': encoding},
                     {'random__strategy': strategy}]

In [28]:
# find best QuantileTransformer hyperparameters for Nscore column, default hyperparams are best
best_random_column_tuning(LogisticRegression(max_iter=1000, multi_class='multinomial'), QuantileTransformer(), quant_hyperparams, 10, 5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 8 candidates, totalling 40 fits
QuantileTransformer(copy=True, ignore_implicit_zeros=False, n_quantiles=150,
                    output_distribution='uniform', random_state=None,
                    subsample=100000)


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.9s finished


In [29]:
# find best KBinsDiscretizer hyperparameters for Escore column, strategy='uniform' is best
best_random_column_tuning(LogisticRegression(max_iter=1000, multi_class='multinomial'), KBinsDiscretizer(), kbins_hyperparams, 30, 6)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 11 candidates, totalling 55 fits
KBinsDiscretizer(encode='onehot', n_bins=11, strategy='quantile')


[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    1.7s finished


In [30]:
# find best KBinsDiscretizer hyperparameters for Oscore column, default hyperparameters are best
best_random_column_tuning(LogisticRegression(max_iter=1000, multi_class='multinomial'), KBinsDiscretizer(), kbins_hyperparams, 30, 7)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 11 candidates, totalling 55 fits
KBinsDiscretizer(encode='onehot', n_bins=3, strategy='quantile')


[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    1.8s finished


In [31]:
# find best QuantileTransformer hyperparameters for impulsive column, default hyperparams are best
best_random_column_tuning(LogisticRegression(max_iter=1000, multi_class='multinomial'), QuantileTransformer(), quant_hyperparams, 10, 10)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 8 candidates, totalling 40 fits
QuantileTransformer(copy=True, ignore_implicit_zeros=False, n_quantiles=1000,
                    output_distribution='uniform', random_state=None,
                    subsample=100000)


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.9s finished


In [32]:
# using specific transformers for each column made the performace of the model marginally better in terms of f-1
quant_pipe = Pipeline([('qt', QuantileTransformer())])

min_max_pipe = Pipeline([('mms', MinMaxScaler())])

kbins_pipe = Pipeline([('kbins', KBinsDiscretizer())])

kbins_uniform_pipe = Pipeline([('kbins', KBinsDiscretizer(strategy='uniform'))])

stan_pipe = Pipeline([('ss', StandardScaler())])

continuous_pipe = ColumnTransformer([('quantile', quant_pipe,  ['Nscore', 'Impulsive']),
                                      ('min_max',  min_max_pipe, ['Cscore']),
                                      ('k_bins',  kbins_pipe, ['Oscore']), 
                                      ('k_bins_uniform',  kbins_uniform_pipe, ['Escore']), 
                                      ('standard',  stan_pipe, ['Ascore', 'SS'])])

pipe = Pipeline([('cont_pipe', continuous_pipe), 
                 ('lr', LogisticRegression(max_iter=1000, multi_class='multinomial'))])

lr_cont_model = pipe.fit(X_train_train.iloc[:, 5:], y_train_train)
y_pred = lr_cont_model.predict(X_val.iloc[:, 5:])
print('Balanced Accuracy:', balanced_accuracy_score(y_val, y_pred))
print('F1 weighted:', f1_score(y_val, y_pred, average='weighted'))

Balanced Accuracy: 0.23159267980696555
F1 weighted: 0.24739590681126414


#### Put together categorical and continuous columns <a class="anchor" id="lr_combined_encoders"></a>

In [33]:
# baseline comparison, putting categorical and continuous columns together for predicting
categorical_cols = (X.dtypes == 'object')

categorical_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore'))])

continuous_pipe = Pipeline([('scaler', StandardScaler())])

preprocessing = ColumnTransformer([('categorical', categorical_pipe,  categorical_cols),
                                   ('continuous',  continuous_pipe, ~categorical_cols)])

pipe = Pipeline([('preprocessing', preprocessing), 
                 ('lr', LogisticRegression(max_iter=1000, multi_class='multinomial'))])

fit_predict_pipe(X_train_train, y_train_train, X_val, y_val, pipe)

Balanced Accuracy: 0.29748756247203456
F1 weighted: 0.33522357656205837


In [34]:
# using specific transformers for columns hurt the model
quant_pipe = Pipeline([('qt', QuantileTransformer())])

min_max_pipe = Pipeline([('mms', MinMaxScaler())])

kbins_pipe = Pipeline([('kbins', KBinsDiscretizer())])

kbins_uniform_pipe = Pipeline([('kbins', KBinsDiscretizer(strategy='uniform'))])

stan_pipe = Pipeline([('ss', StandardScaler())])

preprocessing = ColumnTransformer([('ohe',  OneHotEncoder(), categorical_cols), 
                                   ('quantile', quant_pipe,  ['Nscore', 'Impulsive']),
                                   ('min_max',  min_max_pipe, ['Cscore']),
                                   ('k_bins',  kbins_pipe, ['Oscore']), 
                                   ('k_bins_uniform',  kbins_uniform_pipe, ['Escore']), 
                                   ('standard',  stan_pipe, ['Ascore', 'SS'])])

pipe = Pipeline([('preprocessing', preprocessing), 
                 ('lr', LogisticRegression(max_iter=1000, multi_class='multinomial'))])

fit_predict_pipe(X_train_train, y_train_train, X_val, y_val, pipe)

Balanced Accuracy: 0.2915788519049389
F1 weighted: 0.32972758853673834


#### Search for best hyperparameters <a class="anchor" id="lr_hyperparameters"></a>

In [35]:
# before hyperparameter tuning
categorical_cols = (X.dtypes == 'object')

categorical_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore'))])

continuous_pipe = Pipeline([('scaler', StandardScaler())])

preprocessing = ColumnTransformer([('categorical', categorical_pipe,  categorical_cols),
                                   ('continuous',  continuous_pipe, ~categorical_cols)])

pipe = Pipeline([('preprocessing', preprocessing), 
                 ('random', LogisticRegression(max_iter=1000, multi_class='multinomial'))])

fit_predict_pipe(X_train_train, y_train_train, X_val, y_val, pipe)

Balanced Accuracy: 0.29748756247203456
F1 weighted: 0.33522357656205837


In [36]:
# grid for LogisticRegression
penalty = ['l1', 'l2', 'none']
tol = [float(x) for x in np.logspace(-1, -7, 7)]
C = [float(x) for x in np.logspace(3, -3, 7)]
fit_intercept = [True, False]
class_weight = ['balanced', None]
solver = ['newton-cg', 'lbfgs', 'sag', 'saga']    # liblinear does not support multinomial

hyperparameters = [{'random': [LogisticRegression(multi_class='multinomial', max_iter=1000)],
                    'random__penalty': penalty,
                    'random__tol': tol,
                    'random__C': C,
                    'random__fit_intercept': fit_intercept,
                    'random__class_weight': class_weight,
                    'random__solver': solver}]

In [ ]:
# tuning hyperparameters hurt the model
fit_and_predict_random(hyperparameters, 1000)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.3min


Tuning the hyperparameters actually hurt the model performance on the validation set. The default hyperparameters provided by Sci-kit learn are better for predicting the data.

### RandomForest <a class="anchor" id="RandomForest"></a>

#### SMOTE <a class="anchor" id="rf_smote"></a>

In [ ]:
# baseline RandomForest performance without SMOTE
categorical_cols = (X.dtypes == 'object')

categorical_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore'))])

continuous_pipe = Pipeline([('scaler', StandardScaler())])

preprocessing = ColumnTransformer([('categorical', categorical_pipe,  categorical_cols),
                                   ('continuous',  continuous_pipe, ~categorical_cols)])

pipe = Pipeline([('preprocess', preprocessing),
                 ('rf', RandomForestClassifier())])

fit_predict_pipe(X_train_train, y_train_train, X_val, y_val, pipe)

In [ ]:
# try adding SMOTE to RandomForest
pipe = Pipeline([('preprocess', preprocessing),
                 ('random', SMOTE()),
                 ('rf', RandomForestClassifier())])

In [ ]:
# hyperparameter grid for SMOTE
strategy = ['minority', 'not minority', 'not majority', 'all', 'auto']
neighbors = [1, 3, 5, 7, 9]

hyperparameters = [{'random': [SMOTE()],
                    'random__sampling_strategy': strategy,
                    'random__k_neighbors': neighbors}]

In [ ]:
# adding SMOTE seems to help RandomForest, using k_neighbors=3, sampling_strategy='not majority'
fit_and_predict_random(hyperparameters, 25)

The concern about imbalanced data doesn't seem to affect the models, they make better predictions without considering it.

#### Column Transformations <a class="anchor" id="rf_encoding"></a>

##### Categorical columns <a class="anchor" id="rf_categorical"></a>

In [ ]:
# baseline prediction for RandomForest only using categorical columns
pipe = Pipeline([('ohe', OneHotEncoder()), 
                 ('smote', imblearn.over_sampling.SMOTE(k_neighbors=3, sampling_strategy='not minority')),
                 ('rf', RandomForestClassifier())])

rf_cat_model = pipe.fit(X_train_train.iloc[:, :4], y_train_train)
y_pred = rf_cat_model.predict(X_val.iloc[:, :4])
print('Balanced Accuracy:', balanced_accuracy_score(y_val, y_pred))
print('F1 weighted:', f1_score(y_val, y_pred, average='weighted'))

In [ ]:
# possible category encoders
cat_encoders = [{'random': [CatBoostEncoder()]}, 
                {'random': [GLMMEncoder()]}, 
                {'random': [HashingEncoder()]},  
                {'random': [JamesSteinEncoder()]}, 
                {'random': [LeaveOneOutEncoder()]}, 
                {'random': [MEstimateEncoder()]}, 
                {'random': [OneHotEncoder()]}, 
                {'random': [OrdinalEncoder()]}, 
                {'random': [TargetEncoder()]}, 
                {'random': [WOEEncoder()]}]

In [ ]:
# find best encoder for age column
best_random_column(RandomForestClassifier(), cat_encoders, 10, 0)
# find best encoder for education column
best_random_column(RandomForestClassifier(), cat_encoders, 10, 2)
# find best encoder for country column
best_random_column(RandomForestClassifier(), cat_encoders, 10, 3)
# find best encoder for ethnicity column
best_random_column(RandomForestClassifier(), cat_encoders, 10, 4)

In [ ]:
# glm encoder is best for all 
glm_pipe = Pipeline([('glm', GLMMEncoder())])

# since gender is binary in this dataset, doesn't need anything fancy
one_hot_pipe = Pipeline([('ohe', OneHotEncoder())])

categorical_pipe = ColumnTransformer([('other_cats', glm_pipe,  ['Age', 'Education', 'Country', 'Ethnicity']),
                                      ('gender',  one_hot_pipe, ['Gender'])])

pipe = Pipeline([('cat_pipe', categorical_pipe), 
                 ('smote', imblearn.over_sampling.SMOTE(k_neighbors=3, sampling_strategy='not minority')),
                 ('rf', RandomForestClassifier())])

# using specific encoders for each column hurt the model performance
fit_predict_pipe(X_train_train, y_train_train, X_val, y_val, pipe)

##### Numeric columns <a class="anchor" id="rf_continuous"></a>

In [ ]:
# baseline prediction performance before specifically encoding each column
pipe = Pipeline([('ss', StandardScaler()), 
                 ('smote', imblearn.over_sampling.SMOTE(k_neighbors=3, sampling_strategy='not minority')),
                 ('rf', RandomForestClassifier())])

rf_cont_model = pipe.fit(X_train_train.iloc[:, 5:], y_train_train)
y_pred = rf_cont_model.predict(X_val.iloc[:, 5:])
print('Balanced Accuracy:', balanced_accuracy_score(y_val, y_pred))
print('F1 weighted:', f1_score(y_val, y_pred, average='weighted'))

In [ ]:
# find best encoder for Nscore column
best_random_column(RandomForestClassifier(), cont_encoders, 6, 5)
# find best encoder for Escore column
best_random_column(RandomForestClassifier(), cont_encoders, 6, 6)
# find best encoder for Oscore column
best_random_column(RandomForestClassifier(), cont_encoders, 6, 7)
# find best encoder for Ascore column
best_random_column(RandomForestClassifier(), cont_encoders, 6, 8)
# find best encoder for Cscore column
best_random_column(RandomForestClassifier(), cont_encoders, 6, 9)
# find best encoder for impulsive column
best_random_column(RandomForestClassifier(), cont_encoders, 6, 10)
# find best encoder for SS column
best_random_column(RandomForestClassifier(), cont_encoders, 6, 11)

In [ ]:
# find best QuantileTransformer hyperparameters for Oscore column, default hyperparams are best
best_random_column_tuning(RandomForestClassifier(), QuantileTransformer(), quant_hyperparams, 10, 7)

In [ ]:
# find best QuantileTransformer hyperparameters for Cscore column, default hyperparams are best
best_random_column_tuning(RandomForestClassifier(), QuantileTransformer(), quant_hyperparams, 10, 9)

In [ ]:
# find best QuantileTransformer hyperparameters for impulsive column, n_quantiles=50 is best
best_random_column_tuning(RandomForestClassifier(), QuantileTransformer(), quant_hyperparams, 10, 10)

In [ ]:
# using specific encoders for each column made the performace of the model better
stan_pipe = Pipeline([('ss', StandardScaler())])

min_max_pipe = Pipeline([('mms', MinMaxScaler())])

quant_pipe = Pipeline([('quantile', QuantileTransformer())])

quant_50_pipe = Pipeline([('quantile', QuantileTransformer(n_quantiles=50))])

continuous_pipe = ColumnTransformer([('standard',  stan_pipe, ['Nscore', 'Ascore', 'SS']), 
                                     ('min_max',  min_max_pipe, ['Escore']), 
                                     ('quantile',  quant_pipe, ['Oscore', 'Cscore']),
                                     ('quantile_50',  quant_50_pipe, ['Impulsive'])])

pipe = Pipeline([('cont_pipe', continuous_pipe), 
                 ('smote', imblearn.over_sampling.SMOTE(k_neighbors=3, sampling_strategy='not minority')),
                 ('etc', ExtraTreesClassifier())])

etc_cont_model = pipe.fit(X_train_train.iloc[:, 5:], y_train_train)
y_pred = etc_cont_model.predict(X_val.iloc[:, 5:])
print('Balanced Accuracy:', balanced_accuracy_score(y_val, y_pred))
print('F1 weighted:', f1_score(y_val, y_pred, average='weighted'))

#### Combine categorical and continuous columns back together <a class="anchor" id="rf_combined_encoders"></a>

In [ ]:
# baseline comparison, putting categorical and continuous columns together for predicting
categorical_cols = (X.dtypes == 'object')

categorical_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore'))])

continuous_pipe = Pipeline([('scaler', StandardScaler())])

preprocessing = ColumnTransformer([('categorical', categorical_pipe,  categorical_cols),
                                   ('continuous',  continuous_pipe, ~categorical_cols)])

pipe = Pipeline([('preprocessing', preprocessing), 
                 ('smote', imblearn.over_sampling.SMOTE(k_neighbors=3, sampling_strategy='not minority')),
                 ('rf', RandomForestClassifier())])

fit_predict_pipe(X_train_train, y_train_train, X_val, y_val, pipe)

In [ ]:
# using specific transformers for columns helped the model slightly
stan_pipe = Pipeline([('ss', StandardScaler())])

min_max_pipe = Pipeline([('mms', MinMaxScaler())])

quant_pipe = Pipeline([('quantile', QuantileTransformer())])

quant_50_pipe = Pipeline([('quantile', QuantileTransformer(n_quantiles=50))])

preprocessing = ColumnTransformer([('ohe',  OneHotEncoder(), categorical_cols), 
                                   ('standard',  stan_pipe, ['Nscore', 'Ascore', 'SS']), 
                                   ('min_max',  min_max_pipe, ['Escore']), 
                                   ('quantile',  quant_pipe, ['Oscore', 'Cscore']),
                                   ('quantile_50',  quant_50_pipe, ['Impulsive'])])

pipe = Pipeline([('preprocessing', preprocessing),
                 ('smote', imblearn.over_sampling.SMOTE(k_neighbors=3, sampling_strategy='not minority')),
                 ('rf', RandomForestClassifier())])

fit_predict_pipe(X_train_train, y_train_train, X_val, y_val, pipe)

#### Search for best hyperparameters <a class="anchor" id="rf_hyperparameters"></a>

In [ ]:
# before hyperparameter tuning
stan_pipe = Pipeline([('ss', StandardScaler())])

min_max_pipe = Pipeline([('mms', MinMaxScaler())])

quant_pipe = Pipeline([('quantile', QuantileTransformer())])

quant_50_pipe = Pipeline([('quantile', QuantileTransformer(n_quantiles=50))])

preprocessing = ColumnTransformer([('ohe',  OneHotEncoder(), categorical_cols), 
                                   ('standard',  stan_pipe, ['Nscore', 'Ascore', 'SS']), 
                                   ('min_max',  min_max_pipe, ['Escore']), 
                                   ('quantile',  quant_pipe, ['Oscore', 'Cscore']),
                                   ('quantile_50',  quant_50_pipe, ['Impulsive'])])

pipe = Pipeline([('preprocessing', preprocessing), 
                 ('smote', imblearn.over_sampling.SMOTE(k_neighbors=3, sampling_strategy='not minority')),
                 ('random', RandomForestClassifier())])

fit_predict_pipe(X_train_train, y_train_train, X_val, y_val, pipe)

In [ ]:
# grid for RandomForest
max_depth = [int(x) for x in np.linspace(25, 75, 3)]
max_depth.append(None)
min_samples_split = [int(x) for x in np.linspace(2, 10, 5)]
min_samples_leaf = [int(x) for x in np.linspace(1, 9, 5)]
max_features = ['auto', 'sqrt', 'log2', None]
bootstrap = [True, False]
class_weight = ['balanced', 'balanced_subsample', None]

hyperparameters = [{'random': [RandomForestClassifier()],
                    'random__max_depth': max_depth,
                    'random__min_samples_split': min_samples_split,
                    'random__min_samples_leaf': min_samples_leaf,
                    'random__max_features': max_features,
                    'random__bootstrap': bootstrap,
                    'random__class_weight' : class_weight}]

In [ ]:
# tuning hyperparameters hurt the model
fit_and_predict_random(hyperparameters, 1000)

Again, tuning the hyperparameters hurt the model's performance on the validation set. The default values provided by Sci-kit learn are better for predicting the data.

### Final Model <a class="anchor" id="Final_model"></a>

For my final model, I am choosing the RandomForest model with tuned steps for preprocessing the numeric columns along with incorporating SMOTE into my pipeline.

In [ ]:
# see how my model performs on the test data!
stan_pipe = Pipeline([('ss', StandardScaler())])

min_max_pipe = Pipeline([('mms', MinMaxScaler())])

quant_pipe = Pipeline([('quantile', QuantileTransformer())])

quant_50_pipe = Pipeline([('quantile', QuantileTransformer(n_quantiles=50))])

preprocessing = ColumnTransformer([('ohe',  OneHotEncoder(), categorical_cols), 
                                   ('standard',  stan_pipe, ['Nscore', 'Ascore', 'SS']), 
                                   ('min_max',  min_max_pipe, ['Escore']), 
                                   ('quantile',  quant_pipe, ['Oscore', 'Cscore']),
                                   ('quantile_50',  quant_50_pipe, ['Impulsive'])])

pipe = Pipeline([('preprocessing', preprocessing), 
                 ('smote', SMOTE(k_neighbors=3, sampling_strategy='not minority')),
                 ('random', RandomForestClassifier())])

fit_predict_pipe(X_train, y_train, X_test, y_test, pipe)

Using my tuned and fully trained model, I am able to achieve a F1-weighted of 0.413 on the test data which is about ~0.07 better than my basic original model performed on the validation data. Tuning the preprocessing steps and including SMOTE in my pipeline improved my ability to predict with a random forest model.

### Conclusion <a class="anchor" id="Conclusion"></a>
In the end, a RandomForest model with the default hyperparameters performed the best on the validation set and yielded an F1-weighted score of 0.413 and a balanced accuracy score of 0.315. Before tuning my preprocessing steps and adding SMOTE to my pipeline, my F1-weighted score was 0.348 and my balanced accuracy score was 0.294. 

All the categorical columns performed best with one-hot encoding, which makes sense with respsect to the distribution of labels given the different levels of each categorical variable. For each categorical variable, the mean level of consumption differed for each level of the variable itself.

For the most part, the personality scores were roughly normally distributed, but transforming each column with its ideal transformer yielded better results than just using the StandardScaler on all personality scores.

For the ideal SMOTE parameters, the RandomizedSearchCV chose the best strategy as 'not minority', which may be an artifact of the scoring criteria used by the RandomSearchCV. Since RandomSearchCV was trying to maximize the F1-weighted score, perhaps it decided that predicting the minority class, class 4 ("Used in the last month"), was too difficult to predict, so instead it sampled all other classes in an effort to try and increase F1-weighted score.

Tuning the hyperparameters yielded worse a balanced accuracy score and a worse F1-weighted score than the default hyperparameters. I think this has to do with the overall difficulty of classifying the data. There are 7 labels the classifier is trying to predict and the reality of the distances between the labels aren't very far apart. For instance, is a cannabis user who last used cannabis a week ago versus a month ago that simple to differentiate? I think this is a very difficult prediction to make, which resulted in a lot of misclassifications. 

A better classification problem may be to predict whether someone has used a drug before or has never used a drug before. A model that could accurately predict between those two classes would be more useful in a business context as well.

#### Limitations
It's also important to point out the limitations of this data. Based on the demographic information collected in this dataset, a majority of the participants are between the ages of 18-34 years old, are white, are in either the UK or the USA, and have some college experience. Based on the demographic information, I would not try to use a model trained on this data to try and predict drug consumption for the general population. The demographics of this dataset are not well distributed and therefore, are probably not representative of the population as a whole. 

### Future Directions <a class="anchor" id="Future_directions"></a>
Along with trying a modified classification problem, future work could look into analyzing the features in the dataset to see if there is mulitcollinearity in the data and if so, remove some highly correlated features.

Assuming the simplified classification problem yielded better results, one could also look into the feature importances of the attributes provided in the dataset to, again, try and remove some weakly-predictive features and obtain a simpler, more accurate model.

A final idea I had was to create an overall personality column that classifies participants into certain personality labels based on their personality trait scores. For example, high impulsivity and high sensation seeking could correspond to an "addictive" personality and that attribute could be used to predict drug consumption, rather than using impulsivity and sensation seeking separately.